In [ ]:
!pip install pydub
!pip install librosa

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pydub
from scipy import signal
import librosa
import librosa.display

def get_samples(sound):
    channel_count = sound.channels
    if channel_count == 1:
        samples = np.array(sound.get_array_of_samples())
    elif channel_count == 2:
        sound_mono = sound.split_to_mono()
        samples = np.array(sound_mono[0].get_array_of_samples())
    else:
        raise Exception("More than 2 channels!")
    return samples

def plot_stft(t, f, dB, anchors=None):

    params = {'legend.fontsize': 'x-large','figure.figsize': (25, 6),
         'axes.labelsize': 'x-large','axes.titlesize':'x-large',
         'xtick.labelsize':'x-large','ytick.labelsize':'x-large'}
    plt.rcParams.update(params)

    fig = plt.figure()
    plt.pcolormesh(t, f, dB, cmap='YlOrRd', shading='auto')
    fig.suptitle('Short-time Fourier transform (window duration 50ms, hopsize 10 ms)')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (kHz)')
    plt.colorbar(format='%+02.0f dB')
    plt.ylim([0, 5])
    if anchors is not None:
        anchors_t, anchors_f = map(list, zip(*anchors))
        plt.scatter(anchors_t, anchors_f, marker='x', color='black')
    plt.show()
    
def generate_anchors(t, f, dB):
    delta_t = 0.1
    f_num_bands = 25

    t_rate = len(t) / (t[-1] - t[0])
    delta_i = int(delta_t * t_rate)
    if delta_i == 0:
        delta_i = 1
    delta_j = int(len(f) / f_num_bands)

    anchors_t = []
    anchors_f = []
    anchors = []

    for i in range(0, len(t) - delta_i, delta_i):
        for j in range(0, len(f) - delta_j, delta_j):
            tmp = dB[j:j+delta_j, i:i+delta_i]
            ind = np.unravel_index(np.argmax(tmp, axis=None), tmp.shape)
            anchors_t.append(t[i + ind[1]])
            anchors_f.append(f[j + ind[0]])
            anchors.append((t[i + ind[1]], f[j + ind[0]]))
    
    return anchors

def generate_fingerprint(anchors):
    F = []
    t_offset = 0.1
    t_window = 0.5
    f_factors = (2**-0.5, 2**0.5)

    anchors.sort(key=lambda x: x[0])
    for t1, f1 in anchors:
        t_min = t1 + t_offset
        t_max = t1 + t_offset + t_window
        f_min = f1 * f_factors[0]
        f_max = f1 * f_factors[1]
        for t2, f2 in anchors:
            if t2 < t_min or t2 > t_max or f2 < f_min or f2 > f_max:
                continue
            h = (f1, f2, t2 - t1)
            F.append((t1, h))
    
    return F

def generate_matches(F1, F2):
    matches = []
    h_range = range(len(F1[0][1]))
    for t1, h1 in F1:
        for t2, h2 in F2:
            match = True
            for i in h_range:
                if h1[i] != h2[i]:
                    match = False
                    break
            if match:
                matches.append((t1, t2))
    return matches

def generate_num_of_matches(F1, F2):
    num_of_matches = 0
    h_range = range(len(F1[0][1]))
    for t1, h1 in F1:
        for t2, h2 in F2:
            match = True
            for i in h_range:
                if h1[i] != h2[i]:
                    match = False
                    break
            if match:
                num_of_matches = num_of_matches + 1
    return num_of_matches

def plot_matches(matches, self_compare=False):
    params = {'legend.fontsize': 'x-large','figure.figsize': (12, 12),
            'axes.labelsize': 'x-large','axes.titlesize':'x-large',
            'xtick.labelsize':'x-large','ytick.labelsize':'x-large'}
    plt.rcParams.update(params)
    fig = plt.figure()
    matches_t1, matches_t2 = map(list, zip(*matches))
    plt.scatter(matches_t1, matches_t2, marker='x', color='black')
    plt.xlabel('Time in query signal (s)')
    if self_compare:
        plt.ylabel('Time in query signal (s)')
    else:
        plt.ylabel('Time in document signal (s)')
    plt.show()

E:\Program Files\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def get_stft(snd_file, type = 0):
    if type == 0: sound = pydub.AudioSegment.from_file(snd_file, format="wav") 
    if type == 1: sound = pydub.AudioSegment.from_file(snd_file, format="m4a") 
    samples = get_samples(sound)
    duration = sound.duration_seconds
    window_ms = 50.0
    hop_ms = 10.0
    nperseg = (window_ms / 1000.0) * sound.frame_rate
    noverlap = ((window_ms - hop_ms) / 1000.0) * sound.frame_rate
    f, t, Zxx  = signal.stft(samples, fs = sound.frame_rate, window = 'hann', nperseg = nperseg, noverlap = noverlap, nfft = 50*4/1000*sound.frame_rate)
    f /= 1000.0
    dB = 20 * np.log10(np.abs(Zxx / abs(Zxx).max()))
    return t, f, dB

In [3]:
def get_fingerprint(snd_file, type = 0):
    t, f, dB = get_stft(snd_file, type)
    anchors = generate_anchors(t, f, dB)
    Fp = generate_fingerprint(anchors)
    return Fp

In [4]:
def generate_num_of_matches_new(F1, F2): # the old one was soooo slow 
    F1_hashes = np.array(F1)[:,1]
    F2_hashes = np.array(F2)[:,1]
    n = np.intersect1d(F1_hashes, F2_hashes).shape[0]
    return n

In [5]:
import pickle

In [7]:
f = open('query_5s.pckl', 'wb')
F_q_arr = []
for i in range(1,7):
    if i == 1: query_file = 'query_5s_bouba_0001.wav'
    if i == 2: query_file = 'query_5s_bouba_0277.wav'
    if i == 3: query_file = 'query_5s_bouba_0301.wav'  
    if i == 4: query_file = 'query_5s_kiki_0101.wav'
    if i == 5: query_file = 'query_5s_kiki_0375.wav'
    if i == 6: query_file = 'query_5s_kiki_0500.wav'      
    F_q = get_fingerprint(query_file)
    print("fingerprint generated: Q" + str(i))
    F_q_arr.append(F_q)
pickle.dump(F_q_arr, f)
f.close()

fingerprint generated: Q1
fingerprint generated: Q2
fingerprint generated: Q3
fingerprint generated: Q4
fingerprint generated: Q5
fingerprint generated: Q6


In [8]:
f = open('query_5s.pckl', 'rb')
F_q_arr_read = pickle.load(f)
f.close()
len(F_q_arr_read)

6

In [9]:
F_q = F_q_arr_read[0]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [10]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0001
41689.0


In [11]:
F_q = F_q_arr_read[1]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [12]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0277
43038.0


In [13]:
F_q = F_q_arr_read[2]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [14]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0301
42996.0


In [15]:
F_q = F_q_arr_read[3]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [16]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0101
43200.0


In [17]:
F_q = F_q_arr_read[4]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [18]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0375
43488.0


In [19]:
F_q = F_q_arr_read[5]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [20]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0500
43596.0


In [6]:
f = open('query_1s.pckl', 'wb')
F_q_arr = []
for i in range(1,7):
    if i == 1: query_file = 'query_1s_bouba_0047.wav'
    if i == 2: query_file = 'query_1s_bouba_0119.wav'
    if i == 3: query_file = 'query_1s_bouba_0423.wav'  
    if i == 4: query_file = 'query_1s_kiki_0038.wav'
    if i == 5: query_file = 'query_1s_kiki_0245.wav'
    if i == 6: query_file = 'query_1s_kiki_0361.wav'      
    F_q = get_fingerprint(query_file)
    print("fingerprint generated: Q" + str(i))
    F_q_arr.append(F_q)
pickle.dump(F_q_arr, f)
f.close()

fingerprint generated: Q1
fingerprint generated: Q2
fingerprint generated: Q3
fingerprint generated: Q4
fingerprint generated: Q5
fingerprint generated: Q6


In [7]:
f = open('query_1s.pckl', 'rb')
F_q_arr_read = pickle.load(f)
f.close()
len(F_q_arr_read)

6

In [8]:
F_q = F_q_arr_read[0]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [9]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0047
185.0


In [10]:
F_q = F_q_arr_read[1]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [11]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0119
298.0


In [12]:
F_q = F_q_arr_read[2]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [13]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0423
5990.0


In [14]:
F_q = F_q_arr_read[3]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [15]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0038
236.0


In [16]:
F_q = F_q_arr_read[4]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [17]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0245
5787.0


In [18]:
F_q = F_q_arr_read[5]

max_num_of_matches = np.zeros(50)
max_index = np.zeros(50)
bouba_kiki = np.zeros(50) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(50):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-4-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-4-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [19]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0361
5912.0
